In [12]:
from __future__ import absolute_import
import aiida

In [13]:
from aiida import load_profile
load_profile()

# Import commonly used functionality
import numpy as np
from aiida import orm, engine, common
from aiida.plugins import WorkflowFactory
from aiida.orm import Code



In [14]:
!reentry scan

In [15]:
!verdi daemon restart
#!$JUPYTER_PATH
#!export JUPYTER_PATH="/home/aakhtar/Projects/aiida-defects-siesta/siesta-defect-formation/aiida-workflow/formation_energy/;$JUPYTER_PATH"

Restarting the daemon... OK


In [17]:
from aiida_siesta.workflows.defect_formation.formation_energy_siesta import FormationEnergyWorkchainSIESTA
#import .formation_energy_siesta
#!pwd

In [18]:
from aiida.orm import StructureData
#Structure Pure
cell = [[15.0, 0.0, 0.0,],
        [ 0.0,15.0, 0.0,],
        [ 0.0, 0.0,15.0,],
        ]
pure = StructureData(cell=cell)
pure.append_atom(position=( 0.000 , 0.000 , 0.000 ),symbols=['O']) #1
pure.append_atom(position=( 0.757 , 0.586 , 0.000 ),symbols=['H']) #2
pure.append_atom(position=(-0.757 , 0.586 , 0.000),symbols=['H']) #3 
pure.append_atom(position=( 0.000 , 3.500 , 0.000),symbols=['O']) #4
pure.append_atom(position=( 0.757 , 2.914 , 0.000 ),symbols=['H']) #5
pure.append_atom(position=(-0.757 , 2.914 , 0.000),symbols=['H']) #6


defect=StructureData(cell=cell)
defect.append_atom(position=( 0.000 , 0.000 , 0.000 ),symbols=['O']) #1
defect.append_atom(position=( 0.757 , 0.586 , 0.000 ),symbols=['H']) #2
defect.append_atom(position=(-0.757 , 0.586 , 0.000),symbols=['H']) #3 
defect.append_atom(position=( 0.000 , 3.500 , 0.000),symbols=['O']) #4
defect.append_atom(position=( 0.757 , 2.914 , 0.000 ),symbols=['H']) #5
defect.append_atom(position=(-0.757 , 2.914 , 0.000),symbols=['H'],name="GhostH") #6

In [19]:
code = Code.get_from_string('siesta-psml-lua@N552VW')
charge=-2

In [20]:
from aiida.orm import Dict
parameters_host = Dict(dict={
   "mesh-cutoff": "250 Ry",
   "dm-tolerance": "0.0001",
   "MD-TypeOfRun":   "LUA",
   "LUA-script":   "neb.lua",
   "DM-NumberPulay ":  "3",
   "DM-History-Depth":  "0",
   "SCF-Mixer-weight":  "0.02",
   "SCF-Mix":   "density",
   "SCF-Mixer-kick":  "35",
   "MD-VariableCell":  "F",
   "MD-MaxCGDispl":  "0.3 Bohr",
   "MD-MaxForceTol":  " 0.04000 eV/Ang", 
    })
parameters_defect_q0 = Dict(dict={
   "mesh-cutoff": "250 Ry",
   "dm-tolerance": "0.0001",
   "MD-TypeOfRun":   "LUA",
   "LUA-script":   "neb.lua",
   "DM-NumberPulay ":  "3",
   "DM-History-Depth":  "0",
   "SCF-Mixer-weight":  "0.02",
   "SCF-Mix":   "density",
   "SCF-Mixer-kick":  "35",
   "MD-VariableCell":  "F",
   "MD-MaxCGDispl":  "0.3 Bohr",
   "MD-MaxForceTol":  " 0.04000 eV/Ang", 
   "NetCharge": "0",
    })
parameters_defect_q = Dict(dict={
   "mesh-cutoff": "250 Ry",
   "dm-tolerance": "0.0001",
   "MD-TypeOfRun":   "LUA",
   "LUA-script":   "neb.lua",
   "DM-NumberPulay ":  "3",
   "DM-History-Depth":  "0",
   "SCF-Mixer-weight":  "0.02",
   "SCF-Mix":   "density",
   "SCF-Mixer-kick":  "35",
   "MD-VariableCell":  "F",
   "MD-MaxCGDispl":  "0.3 Bohr",
   "MD-MaxForceTol":  " 0.04000 eV/Ang",
   "NetCharge": str(charge),
    })
options_host=Dict(
    dict={
        "max_wallclock_seconds": 360,
        #'withmpi': True,
        #'account': "tcphy113c",
        #'queue_name': "DevQ",
        "resources": {
            "num_machines": 1,
            "num_mpiprocs_per_machine": 1,
        }
    }
)
options_defect_q0=Dict(
    dict={
        "max_wallclock_seconds": 360,
        #'withmpi': True,
        #'account': "tcphy113c",
        #'queue_name': "DevQ",
        "resources": {
            "num_machines": 1,
            "num_mpiprocs_per_machine": 1,
        }
    }
)
options_defect_q=Dict(
    dict={
        "max_wallclock_seconds": 360,
        #'withmpi': True,
        #'account': "tcphy113c",
        #'queue_name': "DevQ",
        "resources": {
            "num_machines": 1,
            "num_mpiprocs_per_machine": 1,
        }
    }
)



In [21]:
basis_dict_host =Dict(dict= {
'pao-basistype':'split',
'pao-splitnorm': 0.150,
'pao-energyshift': '0.020 Ry',
'%block pao-basis-sizes':
"""
GhostH DZP
H    DZP
%endblock pao-basis-sizes""",
})
basis_dict_defect_q0 = Dict(dict= {
'pao-basistype':'split',
'pao-splitnorm': 0.150,
'pao-energyshift': '0.020 Ry',
'%block pao-basis-sizes':
"""
GhostH DZP
H    DZP
%endblock pao-basis-sizes""",
})
basis_dict_defect_q =Dict(dict= {
'pao-basistype':'split',
'pao-splitnorm': 0.150,
'pao-energyshift': '0.020 Ry',
'%block pao-basis-sizes':
"""
GhostH DZP
H    DZP
%endblock pao-basis-sizes""",
})

In [22]:
parameters_defect_q0.get_dict()

{'mesh-cutoff': '250 Ry',
 'dm-tolerance': '0.0001',
 'MD-TypeOfRun': 'LUA',
 'LUA-script': 'neb.lua',
 'DM-NumberPulay ': '3',
 'DM-History-Depth': '0',
 'SCF-Mixer-weight': '0.02',
 'SCF-Mix': 'density',
 'SCF-Mixer-kick': '35',
 'MD-VariableCell': 'F',
 'MD-MaxCGDispl': '0.3 Bohr',
 'MD-MaxForceTol': ' 0.04000 eV/Ang',
 'NetCharge': '0'}

In [23]:
kpoints_host = orm.KpointsData()
kpoints_host.set_kpoints_mesh([1,1,1]) # Definately not converged, but we want the example to run quickly
kpoints_defect_q0 = orm.KpointsData()
kpoints_defect_q0.set_kpoints_mesh([1,1,1]) # Definately not converged, but we want the example to run quickly
kpoints_defect_q = orm.KpointsData()
kpoints_defect_q.set_kpoints_mesh([1,1,1]) # Definately not converged, but we want the example to run quickly

In [24]:
import os
from aiida_siesta.data.psf import PsfData
pseudo_file_to_species_map = [ ("H.psf", ['GhostH','H']),("O.psf", ['O'])]
pseudos_dict = {}
for fname, kinds, in pseudo_file_to_species_map:
      absname = os.path.realpath(os.path.join("./pseudos",fname))
      pseudo, created = PsfData.get_or_create(absname, use_first=True)
      for j in kinds:
              pseudos_dict[j]=pseudo
pseudos_dict

{'GhostH': <PsfData: uuid: ff59cc29-55e8-45a0-a284-f5c3c945765f (pk: 521)>,
 'H': <PsfData: uuid: ff59cc29-55e8-45a0-a284-f5c3c945765f (pk: 521)>,
 'O': <PsfData: uuid: 4e885319-c33a-42fb-ae15-6f923cb84c5a (pk: 3)>}

In [25]:
inputs = {
    # Structures
    'host_structure': pure,
    'defect_structure': defect,
    # Defect information 
    'defect_charge' : orm.Float(-2.),  
    'defect_site' : orm.List(list=[-0.757 , 2.914 , 0.000]),    # Position of the defect in crystal coordinates
    'fermi_level' : orm.Float(0.0),               # Position of the Fermi level, with respect to the valence band maximum      
    'chemical_potential' : orm.Float(250.709), # eV, the chemical potentical of a C atom
    'gaussian_sigma':orm.Float(0.5),
    'correction_scheme' : orm.Str('gaussian'),
    "epsilon":orm.Float(1.0),
    # Computational (chosen code is QE)
    'siesta' : { 'dft': {'supercell_host':{'code': code, 'kpoints': kpoints_host, 'parameters' : parameters_host,
                                           'options':options_host,"basis": basis_dict_host},
                         'supercell_defect_q0':{'code': code, 'kpoints': kpoints_defect_q,'parameters' : parameters_defect_q0,
                                                'options':options_defect_q0,"basis": basis_dict_defect_q0},
                         'supercell_defect_q':{'code': code, 'kpoints': kpoints_defect_q,'parameters' : parameters_defect_q,
                                               'options':options_defect_q,"basis": basis_dict_defect_q}
}}}

In [26]:
inputs

{'host_structure': <StructureData: uuid: 55b12ea0-48ba-4da2-bd39-5cca61525d90 (unstored)>,
 'defect_structure': <StructureData: uuid: d16feacb-8946-4829-ace0-3e12c5132bf8 (unstored)>,
 'defect_charge': <Float: uuid: 93973bb2-ce58-46c8-9b86-446c855e1650 (unstored) value: -2.0>,
 'defect_site': <List: uuid: bb453418-0930-407b-b6d1-e67ec9177d4f (unstored) value: [-0.757, 2.914, 0.0]>,
 'fermi_level': <Float: uuid: 4d6c42ab-e25d-4a57-8c97-c3e2029edc84 (unstored) value: 0.0>,
 'chemical_potential': <Float: uuid: 7f713dc2-3ba0-4e78-8c1f-d67726006cd4 (unstored) value: 250.709>,
 'gaussian_sigma': <Float: uuid: 5ecd7e2e-9bfb-46ef-acf6-1cbc3f2f2367 (unstored) value: 0.5>,
 'correction_scheme': <Str: uuid: c9fd1291-1d8a-4aa6-9de2-3c61bbcc6344 (unstored) value: gaussian>,
 'epsilon': <Float: uuid: 8de173d3-06d0-48a2-a4ff-304cadc9b961 (unstored) value: 1.0>,
 'siesta': {'dft': {'supercell_host': {'code': <Code: Remote code 'siesta-psml-lua' on N552VW, pk: 403, uuid: 872001d8-73fb-4b50-be30-4b947bd

In [27]:
workchain_future = engine.submit(FormationEnergyWorkchainSIESTA, **inputs)

/home/aakhtar/venv/aiida/lib/python3.6/site-packages/aiida/engine/processes/ports.py:111: UserWarning: default of input port `fermi_level` is a `Node` instance, which can lead to unexpected side effects. It is advised to use a lambda instead, e.g.: `default=lambda: orm.Int(5)`.
  warnings.warn(UserWarning(message))  # pylint: disable=no-member


In [28]:
workchain_future

<WorkChainNode: uuid: ea242e21-1145-46d4-a449-3b852e43661b (pk: 950) (aiida.workflows:siesta.formation_energy_siesta)>

In [29]:
!verdi process list -a

  PK  Created    Process label                   Process State     Process status
----  ---------  ------------------------------  ----------------  -----------------------------------
   8  83D ago    SiestaCalculation               ⏹ Finished [0]
  14  83D ago    SiestaCalculation               ⏹ Finished [0]
  15  83D ago    SiestaCalculation               ⏹ Finished [0]
  25  83D ago    SiestaCalculation               ⏹ Finished [0]
  35  83D ago    SiestaCalculation               ⏹ Finished [0]
  45  83D ago    SiestaCalculation               ⏹ Finished [0]
  55  83D ago    SiestaCalculation               ⏹ Finished [0]
  65  83D ago    SiestaCalculation               ⏹ Finished [0]
  75  83D ago    SiestaCalculation               ⏹ Finished [0]
  85  83D ago    SiestaCalculation               ⨯ Excepted        Waiting for transport task: upload
  86  83D ago    SiestaCalculation               ⨯ Excepted        Waiting for transport task: upload
  87  83D ago    SiestaCalculation 

In [ ]:
!verdi process show 908

In [ ]:
!verdi process report 908

In [ ]:
from aiida.orm import load_node
results=load_node('584')

In [ ]:
results.outputs['output_parameters'.get_dict()]

In [ ]:
results.outputs.output_parameters.get_dict()

In [ ]:
code